# Importando as dependências

In [1]:
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np 
from tqdm.notebook import tqdm
import pandas as pd     
import tensorflow as tf

# Definindo diretórios

In [2]:
dataset_dir = os.path.join(os.getcwd(), 'teethes')

dataset_train_dir = os.path.join(dataset_dir, 'train')
dataset_validation_dir = os.path.join(dataset_dir, 'validation')
features_dir = os.path.join(dataset_dir, 'features')

# Criando classe extratora de features
A classe FeatureExtractor irá utilizar um modelo pronto disponibilizado pelo TensorFlow. Tem-se a arquitetura VGG-16 que é um rede neural convulacional que possui 16 camadas.
Mais informações sobre a arquitetura VGG-16 ver em: [VGG-16 Architecture](https://datagen.tech/guides/computer-vision/vgg16/)

## Sobre o modelo
- O modelo deve receber como entrada imagens de tamanho 224x224
- Como ponderação o modelo utiliza o peso imagenet, amplamente utilizado em modelos para classificação de imagens
- No FeatureExtractor estamos coletando a saída da camada FC1

In [3]:
class FeatureExtractor:
    def __init__(self):
        base_model = VGG16(weights='imagenet')
        self.model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    
    def extract(self, img):
        img = img.resize((224, 224))
        img = img.convert('RGB')
        
        image_array = image.img_to_array(img)
        image_array = np.expand_dims(image_array, axis=0)
        image_array = preprocess_input(image_array)
        
        feature = self.model.predict(image_array)[0]

        return feature / np.linalg.norm(feature)

## Gerando uma imagem com modificações randomizadas
A necessidade de gerar uma imagem com algumas modificações surgiu da possibilidade de ser colocado uma imagem nova no database com alguma modificação, como zoom, flip horizontal entre outras, para contornar esse problema decide-se criar uma augmented dataset que além das imagens originais, também possui as imagens originais modificadas

In [14]:
def applyRandomizedModificationInImages(original_image):
    temp_dir = os.path.join(dataset_dir, 'temp')
    
    data_generator = ImageDataGenerator(
        zoom_range=[0.8, 1.2], 
        horizontal_flip=True,  
        rotation_range=30  
    )
    
    expanded_image = tf.expand_dims(original_image, axis=0)
    
    modified_image = data_generator.flow(expanded_image, batch_size=1)[0][0]
    modified_image = Image.fromarray(modified_image.astype('uint8'), 'RGB')
    modified_image.save(os.path.join(temp_dir, f'{original_image.filename}_augmented.png'))
    
    modified_image_path = os.path.join(temp_dir, f'{original_image.filename}_augmented.png')
    
    return modified_image_path

# Obtendo as features das imagens no datasete de treino
Para obter as features das imagens, itera-se sobre cada uma delas existente no dataset de treinamento chamando a função extract() da instância do feature_extractor

In [5]:
feature_extractor = FeatureExtractor()

In [ ]:
dataset_train_images_dir = os.path.join(dataset_train_dir, 'images')

dataset_train_file_images = os.listdir(dataset_train_images_dir)

for image_name in tqdm(dataset_train_file_images):
    feature = feature_extractor.extract(img=Image.open(os.path.join(dataset_train_images_dir, str(image_name))))
    
    feature_path = os.path.join(features_dir, '{}.npy'.format(str(image_name).replace('.png','')))
    np.save(feature_path, feature)

    augmented_image_path = applyRandomizedModificationInImages(Image.open(os.path.join(dataset_train_images_dir, str(image_name))))
    augmented_feature = feature_extractor.extract(img=Image.open(augmented_image_path))
    augmented_feature_path = os.path.join(features_dir, '{}_augmented.npy'.format(str(image_name).replace('.png','')))
    np.save(augmented_feature_path, augmented_feature)

## Colocando as features em um dataframe

In [8]:
features = []
for file_name in tqdm(os.listdir(features_dir)):
  add_feature = np.load(os.path.join(features_dir, str(file_name)))
  add_feature = pd.DataFrame([add_feature])
  
  add_feature['image'] = file_name.replace('.npy','.png')

  features.append(add_feature)

features = pd.concat(features, axis=0)
features.to_csv(os.path.join(dataset_dir, 'feature_extraction.csv'), index=False)

  0%|          | 0/4999 [00:00<?, ?it/s]

## Obtendo as features já salvas

In [12]:
loaded_features = pd.read_csv(os.path.join(dataset_dir, 'feature_extraction.csv'))

# Verificando a semelhança da imagem com o dataset disponível
1. Cópia do conteúdo existente no DataFrame loaded_features
2. Remoção da coluna image(que possui o nome do arquivo da imagem) do DataFrame, já que ela não possui informação relevante para o conteúdo das imagens
3. Obtenção apenas dos valores das features
4. Cálculo da distância euclidiana(responsável em indicar a similiridade entre duas imagens) entre as features das imagens do dataset de treino com as features da imagem fornecida como entrada

In [66]:
def verify_similarity(image_name):
    dataset_manual_tests = os.path.join(dataset_dir, 'manual_tests')

    image_to_compare = Image.open(os.path.join(dataset_manual_tests, image_name))
    image_to_compare_features = feature_extractor.extract(image_to_compare)

    features_data = loaded_features.copy()
    features_data = features_data.drop(columns = ['image'])
    features_data = features_data.values

    euclidean_distance = np.linalg.norm(features_data - image_to_compare_features, axis=1)

    lower_distance_images_quantity = 30
    images_with_lowest_distance_ids = np.argsort(euclidean_distance)[:lower_distance_images_quantity]

    # Get only the lines with the images that have the given id
    more_similar_images = loaded_features.iloc[images_with_lowest_distance_ids,:]['image']
    scores = pd.DataFrame({
        'image': more_similar_images,
        'score': euclidean_distance[images_with_lowest_distance_ids]}
    )
    images_and_scores = scores.reset_index(drop=True)

    return images_and_scores, image_to_compare

## Visualizando as imagens semelhantes a passada pela entrada
Para verificar se uma imagem é semelhante a outra deve-se definir qual nível de semelhança deve ser considerado para classifica-la como "iguais". Como na aplicação atual busca-se imagens repetidas, normalmente a distância euclidiana deve ser bem baixa.

In [71]:
def plot_images_with_similarity(image1, image2, similarity_score):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

  ax1.imshow(image1)
  ax1.set_title('Imagem to compare: ' + os.path.basename(image1.filename))
  ax1.axis('off')

  ax2.imshow(image2)
  ax2.set_title('Imagem in dataset: ' + os.path.basename(image2.filename))
  ax2.axis('off')

  fig.text(0.5, 0.05, 'Similaridade: {:.2f}%'.format(100 - similarity_score), ha='center', fontsize=12)

  plt.tight_layout()
  plt.show()


## Testes manuais feitos
1. Imagem com inversão horizontal
2. Imagem com inversão vertical
3. Imagem com rotação de 90 graus para a direita
4. Imagem com zoom

In [82]:
## 1
# images_and_scores, image_to_compare = verify_similarity('2001_horizontal_revert.png')
## 2
# images_and_scores, image_to_compare = verify_similarity('2001_vertical_revert.png')
## 3
# images_and_scores, image_to_compare = verify_similarity('2001_90_rotation.png')
## 4
images_and_scores, image_to_compare = verify_similarity('2001_zoomed.png')

1/1 [==============================] - 0s 168ms/step


In [88]:
minimum_euclidean_distance_to_be_equal = 0.3

images_and_scores = images_and_scores.sort_values(by=['score'])

for i in range(len(images_and_scores)):
    score = images_and_scores['score'][i]
    if score < minimum_euclidean_distance_to_be_equal:
        plot_images_with_similarity(image_to_compare, Image.open(os.path.join(dataset_train_dir, 'images', images_and_scores['image'][i]).replace('.jpg','')), score)
